- 工作階段要分清楚，request、parse資料的階段要分清楚，比較不容易發生錯誤
- 先用 list 的方式存資料，最後再一起整理成Dataframe
- 塞Cookie可以減少等待的時間，正常直接request都要等5秒
    - 至於Cookie怎麼取可以開Selenium或直接複製一個新的就可以了
- 時間、電腦容量

註：本篇文章僅供研究使用，請勿惡意大量爬取資料造成對方公司的負擔

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from datetime import datetime

In [2]:
headers = {'cookie': 'ECC=GoogleBot',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

query = '筆電'
# 每次加載分頁有20筆資料，可以依需求增加/減少
pages = 50

# get prods list
prodids = []
for page in list(range(1, pages)):
    url = 'https://ecshweb.pchome.com.tw/search/v3.3/all/results?q={}&page={}&sort=sale/dc'.format(query, page)
    resp = requests.get(url,headers=headers)
    for prodid in resp.json()['prods']:
        prodids.append(prodid['Id'])
    prodids = list(set(prodids))
    print('There are {} products in list.'.format(len(prodids)))

# ecapi
df1 = []
for i, Id in enumerate(prodids):
    columns, values = [], []
    print('Dealing with {}: {}'.format(i, Id))
    sleep(0.7)
    ecapi = 'https://mall.pchome.com.tw/ecapi/ecshop/prodapi/v2/prod/{}&fields=Seq,Id,Stmt,Slogan,Name,Nick,Store,PreOrdDate,SpeOrdDate,Price,Discount,Pic,Weight,ISBN,Qty,Bonus,isBig,isSpec,isCombine,isDiy,isRecyclable,isCarrier,isMedical,isBigCart,isSnapUp,isDescAndIntroSync,isFoodContents,isHuge,isEnergySubsidy,isPrimeOnly,isWarranty,isLegalStore,isOnSale,isPriceTask,isBidding,isETicket&_callback=jsonp_prod&1587196620'.format(Id)
    resp = requests.get(ecapi, headers=headers)
    data = re.sub('try{jsonp_prod\(|\}\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\)\;\}','',resp.text)
    data = json.loads(data)[Id+'-000']

    for key, value in data.items():
        columns.append(key)
        values.append(value)
    ndf = pd.DataFrame(data=values, index=columns).T
    df1.append(ndf)
df1 = pd.concat(df1, ignore_index=True)


# cdn
df2 = []
for i, Id in enumerate(prodids):
    columns, values = [], []
    print('Dealing with {}: {}'.format(i, Id))
    sleep(0.7)
    cdn = 'https://ecapi.pchome.com.tw/cdn/ecshop/prodapi/v2/prod/{}/desc&fields=Id,Stmt,Equip,Remark,Liability,Kword,Slogan,Author,Transman,Pubunit,Pubdate,Approve&_callback=jsonp_desc'.format(Id+'-000')
    resp = requests.get(cdn, headers=headers)
    data = re.sub('try\{jsonp_desc\(|\}\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\)\;\}','',resp.text)
    data = json.loads(data)
    data = data[Id]

    for key, value in data.items():
        columns.append(key)
        values.append(value)
    ndf = pd.DataFrame(data=values, index=columns).T
    df2.append(ndf)

df2 = pd.concat(df2, ignore_index=True)

There are 20 products in list.
There are 40 products in list.
There are 60 products in list.
There are 80 products in list.
There are 100 products in list.
There are 120 products in list.
There are 140 products in list.
There are 159 products in list.
There are 179 products in list.
There are 199 products in list.
There are 219 products in list.
There are 239 products in list.
There are 259 products in list.
There are 279 products in list.
There are 299 products in list.
There are 319 products in list.
There are 339 products in list.
There are 359 products in list.
There are 379 products in list.
There are 399 products in list.
There are 419 products in list.
There are 439 products in list.
There are 459 products in list.
There are 479 products in list.
There are 499 products in list.
There are 519 products in list.
There are 539 products in list.
There are 559 products in list.
There are 579 products in list.
There are 599 products in list.
There are 618 products in list.
There are 63

Dealing with 193: DRAC8N-A9009YE3L
Dealing with 194: DHAV5V-A900A8INB
Dealing with 195: DCABIE-A900AICOT
Dealing with 196: DIBMAM-A900ADZMX
Dealing with 197: DCBB69-A900AA99P
Dealing with 198: DRAC7A-A9006WBDK
Dealing with 199: DIBMWZ-A90096H8L
Dealing with 200: DHAFFN-A900AH9D4
Dealing with 201: DHBF5D-A900ANTIB
Dealing with 202: DICM0S-A900AJP1G
Dealing with 203: DCAG8T-A9006WAS5
Dealing with 204: DICM9N-A9008733V
Dealing with 205: DCBB43-A9009XM02
Dealing with 206: DCADJF-A90068S86
Dealing with 207: DICM66-A900AKTV8
Dealing with 208: DICMIR-A900A8YVK
Dealing with 209: DICMI5-A90086SNC
Dealing with 210: DHAEBP-A900ADDYN
Dealing with 211: DHAFK9-1900AIYO1
Dealing with 212: DICMIO-A9008E9SP
Dealing with 213: DHAK5T-A900AL1RG
Dealing with 214: DSAJ2B-A900AHX3T
Dealing with 215: DHAFKI-A900AMCDZ
Dealing with 216: DCAX83-A900AKL3Y
Dealing with 217: DIBMYU-A900AF2WU
Dealing with 218: DCAGT8-A900AEZIA
Dealing with 219: DGBX60-A900AENTF
Dealing with 220: DCBB7J-A900AFHF8
Dealing with 221: DR

Dealing with 428: DCAGTU-A900AMKY3
Dealing with 429: DHAEF7-A900A9L1F
Dealing with 430: DICMIQ-A9009LIMV
Dealing with 431: DHAEBP-A900ANKZJ
Dealing with 432: DCAH55-A9008AXJ6
Dealing with 433: DGAX10-A9007F157
Dealing with 434: DHAFJA-1900AJ43Y
Dealing with 435: DHAFFN-A9009T9IJ
Dealing with 436: DICMHU-A9008U9JT
Dealing with 437: DHAK5U-A900ALCA8
Dealing with 438: DCBB7J-A900AJTE5
Dealing with 439: DHAFJX-A900AO1KB
Dealing with 440: DHAK5Q-A900AMH16
Dealing with 441: DIBM6K-A9008NMUS
Dealing with 442: DICMHB-A900AKZ1G
Dealing with 443: DCAG5D-A9007G1V0
Dealing with 444: DIBMUZ-A900AE6PL
Dealing with 445: DICM4I-A90081W0C
Dealing with 446: DSACPP-A9006NTLD
Dealing with 447: DCAG51-A9005K0HT
Dealing with 448: DRAC7A-A9007IPKP
Dealing with 449: DICMHZ-A900AJD6N
Dealing with 450: DHAV5T-A900AN87V
Dealing with 451: DCABIE-A90086W5Z
Dealing with 452: DHAEBP-A900AOBBC
Dealing with 453: DCACHS-A900AFY15
Dealing with 454: DICM75-A900A1MU5
Dealing with 455: DHAFIY-A900A570E
Dealing with 456: DC

Dealing with 663: DYAF13-A900A8A2X
Dealing with 664: DHAFI3-A900ABRBH
Dealing with 665: DCAH55-A9007I8ZL
Dealing with 666: DHBF2Z-A900AIE2E
Dealing with 667: DCAA40-A90079TV5
Dealing with 668: DIBMW1-A900AE66Q
Dealing with 669: DHAK64-A900AP7UW
Dealing with 670: DGAXA7-A9009Y83P
Dealing with 671: DCBB43-A9009V82P
Dealing with 672: DCADHP-A9007AO55
Dealing with 673: DCAH55-A75468734
Dealing with 674: DHAK8H-1900AO5OQ
Dealing with 675: DCABI0-A9009H4EC
Dealing with 676: DHAV5V-A900AHN7K
Dealing with 677: DRAE7Q-A9005YDXM
Dealing with 678: DCAGSU-A900ANLS5
Dealing with 679: DGBO02-A9005E1UC
Dealing with 680: DHAFJA-1900AJ447
Dealing with 681: DICKDC-A900AF3TA
Dealing with 682: DHAK8H-1900AO5RN
Dealing with 683: DCABIE-A90086SNJ
Dealing with 684: DGAXDA-A900AJ3ZC
Dealing with 685: DCABIE-A900AM50O
Dealing with 686: DHAEBP-A900ADFXQ
Dealing with 687: DHACAC-A900AAHNC
Dealing with 688: DHAFIR-A900AEG0T
Dealing with 689: DGAV0M-A900AP115
Dealing with 690: DHAFJ7-A900ALGCH
Dealing with 691: DC

Dealing with 898: DHAEC9-A900AHM4E
Dealing with 899: DHAV5T-A900AMNH7
Dealing with 900: DCBB7J-A900AFBC1
Dealing with 901: DHAFHQ-A900AMP78
Dealing with 902: DIBMIM-A9008AQ08
Dealing with 903: DICMIW-A900AKMTB
Dealing with 904: DHAFFM-A900AILQV
Dealing with 905: DCADHP-A9006Q817
Dealing with 906: DCAIAM-A9007SHQL
Dealing with 907: DCAD4K-A79894061
Dealing with 908: DHAS1U-A900AMWRG
Dealing with 909: DHAFCU-A9009VYVA
Dealing with 910: DHAS1U-A900AMWRO
Dealing with 911: DICMHU-A900835ZX
Dealing with 912: DIBMKD-A9008Q2HF
Dealing with 913: DHAV5T-A900AOOJJ
Dealing with 914: DHAX94-A900AOR9R
Dealing with 915: DHAFJ7-A900ALGCO
Dealing with 916: DHAX8W-A900AO7SG
Dealing with 917: DEAB8D-A9008QW56
Dealing with 918: DHAFK9-1900AOECN
Dealing with 919: DICMIQ-A900A12MK
Dealing with 920: DICM0S-A900AKMN8
Dealing with 921: DHAFJX-A900AOH3R
Dealing with 922: DCAG7L-A9005ELJX
Dealing with 923: DHACAD-A900A5QO3
Dealing with 924: DGAXC7-A9009FAT2
Dealing with 925: DGAX6J-A900AKZK9
Dealing with 926: DI

Dealing with 158: DHAFHW-A9009GJZ1
Dealing with 159: DHAFJC-1900AH9RR
Dealing with 160: DICM0S-A900AKMNI
Dealing with 161: DIBMMA-A900AJ09G
Dealing with 162: DEAB1J-A9009ALR1
Dealing with 163: DCAGTU-A900AMCY1
Dealing with 164: DHBF5D-A900ANLEI
Dealing with 165: DHAV5T-A900AFWW3
Dealing with 166: DHAFJB-1900AMA9F
Dealing with 167: DCAGOO-A900A5IH5
Dealing with 168: DEAB1J-A9008PSVY
Dealing with 169: DIBM0Y-A9005QU5Y
Dealing with 170: DCBAGG-A900AMD0O
Dealing with 171: DCBB66-A900A0IID
Dealing with 172: DICMIO-A9008B5LP
Dealing with 173: DCAGHK-A90076XP4
Dealing with 174: DHAK8H-1900AO5NQ
Dealing with 175: DICMIW-A900AKMTI
Dealing with 176: DHBF3Y-A900ANLER
Dealing with 177: DCAD2V-A900644G3
Dealing with 178: DHAV5T-A900ANM68
Dealing with 179: DHAEDC-A9009KYMT
Dealing with 180: DHAX8W-1900AHROE
Dealing with 181: DHBF2Z-A900AJ6DU
Dealing with 182: DHAV5V-A900ALTPF
Dealing with 183: DIBMZM-A900AE6B2
Dealing with 184: DRAE7Q-A9005YDXK
Dealing with 185: DYAO7U-A9009GHVW
Dealing with 186: DC

Dealing with 393: DICMIR-A900A8YVJ
Dealing with 394: DCAC2M-A9008W77G
Dealing with 395: DICM0S-A900AKMP4
Dealing with 396: DCADJL-A900A6GAP
Dealing with 397: DHBF2Y-A900ANLEA
Dealing with 398: DICMHZ-A900AKMS0
Dealing with 399: DCAGU1-A900AN1TT
Dealing with 400: DHAK8I-A900AP3JI
Dealing with 401: QAAH0J-A900ALGWK
Dealing with 402: DICM0S-A900AJNLR
Dealing with 403: DHAK64-A900AP40H
Dealing with 404: DICMI5-A9009XMO1
Dealing with 405: DICMIO-A9009I2RU
Dealing with 406: DHAFHQ-A900AMA95
Dealing with 407: DHAFHX-A900A4NG5
Dealing with 408: DHAFK9-1900AOEC8
Dealing with 409: DHAFJB-1900AMA90
Dealing with 410: DEAS6S-A900ABRKT
Dealing with 411: DRAB1Q-A9006ED4F
Dealing with 412: DICMI6-A9009GV77
Dealing with 413: DHBF3I-A9009Y7TS
Dealing with 414: DICM0S-A900AKMNZ
Dealing with 415: DHAFFN-A9008K0AE
Dealing with 416: DEBUD0-A90090R3R
Dealing with 417: DHAFJX-A900AIPNN
Dealing with 418: DEAB7S-A9007K506
Dealing with 419: DICM0S-A900AKMQF
Dealing with 420: DICMI5-A9009I2UY
Dealing with 421: DH

Dealing with 628: DICM66-A900ACE5W
Dealing with 629: DGAV0M-A900ANX1T
Dealing with 630: DCBB66-A9009L7PB
Dealing with 631: DEBUFC-A90099WR6
Dealing with 632: DMAFDK-A90090WE5
Dealing with 633: DCAGSU-A900AASRN
Dealing with 634: DHAFK9-A900ALX5L
Dealing with 635: DCADI4-A9009XQ60
Dealing with 636: DCBB7J-A900AN1SZ
Dealing with 637: DICMHZ-A900AKMQU
Dealing with 638: DICMIH-A900AP0E1
Dealing with 639: DCADHP-A9009484C
Dealing with 640: DGAX6J-A900AL1CH
Dealing with 641: DHAFFN-A9009Z92X
Dealing with 642: DHAEF7-A900A9L0M
Dealing with 643: DICMIW-A900AKMVC
Dealing with 644: DGBK26-A900647UI
Dealing with 645: DEALO4-A9007A9VI
Dealing with 646: DHAFER-A900AN6V5
Dealing with 647: DHAFIZ-A900ALT2E
Dealing with 648: DCAD8H-A9006QM9E
Dealing with 649: DHAEG8-A900AO1X8
Dealing with 650: DCBB66-A9009YTH4
Dealing with 651: DICMIQ-A9008FSPJ
Dealing with 652: DHAFHQ-A900ANUD7
Dealing with 653: DICM0S-A900AJNOZ
Dealing with 654: DICM73-A9008BIZI
Dealing with 655: DHAFHQ-A900ANDKS
Dealing with 656: DI

Dealing with 863: DHACAC-A900A5M15
Dealing with 864: DYAR6U-A9009T6SD
Dealing with 865: DCAC00-A9007B5NX
Dealing with 866: DICMIR-A9009Z8R5
Dealing with 867: DRAE2G-A9008FZEQ
Dealing with 868: DICM0S-A900AKMXO
Dealing with 869: DHAFJB-1900AJ4J0
Dealing with 870: DCADJL-A900AB56C
Dealing with 871: DICM0S-A900AKMXU
Dealing with 872: DHAK8H-1900AO8QW
Dealing with 873: DHAV5V-A900A8INK
Dealing with 874: DCADHP-A90060MSP
Dealing with 875: DICM0S-A900AKMXX
Dealing with 876: DHAK5T-A900AMPXX
Dealing with 877: DICMHU-A90093FXB
Dealing with 878: DEAC9D-A81011500
Dealing with 879: DICM66-A900ANM0C
Dealing with 880: DHAFK9-A900ALX60
Dealing with 881: DYAF69-A9009FSQG
Dealing with 882: DCAGSU-A900AC89O
Dealing with 883: DICMHZ-A900AKMQM
Dealing with 884: DHAFIY-A900AGV1Z
Dealing with 885: DHAFK9-A900AJSLI
Dealing with 886: DHAV5V-A900A7ESS
Dealing with 887: DHAEBH-A900AN6GE
Dealing with 888: DCABLN-A9009HK1R
Dealing with 889: DEAB6F-A900AF5Y9
Dealing with 890: DCBB7J-A900ACOZW
Dealing with 891: DI

In [5]:
df1['Id'] = df1['Id'].apply(lambda x: re.sub('-000$','',x))
df = pd.merge(df1, df2, how='left',on='Id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 0 to 977
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Seq                 978 non-null    object
 1   Id                  978 non-null    object
 2   Name                978 non-null    object
 3   Nick                978 non-null    object
 4   Store               978 non-null    object
 5   PreOrdDate          978 non-null    object
 6   SpeOrdDate          978 non-null    object
 7   Price               978 non-null    object
 8   Discount            978 non-null    object
 9   Pic                 978 non-null    object
 10  Weight              978 non-null    object
 11  ISBN                978 non-null    object
 12  Qty                 978 non-null    object
 13  Bonus               978 non-null    object
 14  isBig               978 non-null    object
 15  isSpec              978 non-null    object
 16  isCombine           978 no

In [7]:
df.to_excel('./Pchome_NB.xlsx')
df

,Seq,Id,Name,Nick,Store,PreOrdDate,SpeOrdDate,Price,Discount,Pic,...,Equip,Remark,Liability,Kword,Slogan,Author,Transman,Pubunit,Pubdate,Approve
0,23378989,DHAK5U-A900A7FA1,MSI微星 GL65 9SD-035TW(i7-9750H/8G/512G SSD/GTX1...,<font color=#FF00CC><b>原價$43900★GTX1660Ti獨顯</f...,DHAK5U,,,"{'M': 0, 'P': 40900, 'Prime': ''}",0,{'B': '/items/DHAK5UA900A7FA1/000001_156617686...,...,"原廠電競級專用後背包,原廠電競滑鼠",,"2年保固,台灣地區2年到府收送\r\n免費客服電話0800-018880",微星^msi,<font color=#663399><b>《 超狂特色。飛炫到不行 》\r\n★搭載最新...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
1,24526905,DHAK5T-A900AKZN9,MSI微星 GP75 10SFK-008TW 黑(i7-10750H/8G/RTX2070-...,<font color=#FF00CC><b>10代i7★RTX2070獨顯</font><...,DHAK5T,,,"{'M': 0, 'P': 56900, 'Prime': ''}",0,{'B': '/items/DHAK5TA900AKZN9/000001_158555567...,...,"原廠電競級專用後背包,原廠電競滑鼠",商品文案中相關圖文及說明.僅供參考.商品規格請以實際出貨為準,"2年保固,台灣地區2年到府收送\r\n免費客服電話0800-018880",微星^msi,<font size=2><LI>處理器：Intel 第10代 Core i7-10750H...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
2,24715004,DYAF13-A900AN756,AHAStyle Macbook 桌上型 鋁合金立式支架 筆電立架 V3鋁合金系列 銀色,AHAStyle Macbook 桌上型 鋁合金立式支架 筆電立架 V3鋁合金系列,DYAF13,,,"{'M': 0, 'P': 890, 'Prime': ''}",0,{'B': '/items/DYAF13A900AN756/000001_158951336...,...,鋁合金筆電支架*1,本商品不含筆電\r\n\r\n台灣總代理 艾海國際有限公司,,Macbook^筆電立架^鋁合金立架^立式支架,*堅固鋁合金材質，支撐更穩\r\n*放置筆電處寬度好調整，可直接手調不需工具輔助\r\n*筆...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
3,22076376,DICMIX-A9009T6UQ,Moshi Pluma 輕薄防震筆電內袋 (MacBook 13),"Moshi Pluma 輕薄防震筆電內袋 (MacBook Pro/Air 13, 2018...",DICMIX,,,"{'M': 1190, 'P': 1190, 'Prime': ''}",0,{'B': '/items/DICMIXA9009T6UQ/000001_155048171...,...,,* 本銷售商品不含電腦主機\r\n* 魚骨紋灰 賣場：https://24h.pchome....,* 進入Moshi 官網註冊登記此產品，即能辨別真偽，並可延長保固：www.moshi.co...,"moshi,macbook保護,macbookpro保護,macbookpro13保護,ma...",■ 專為 USB-C 端口的 MacBook Pro/Air 所設計\r\n■ 加厚軟墊內襯...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
4,22417085,DICMI6-A9009XMU9,14/15.4吋 磨砂PU手提筆電包 電腦包 (DH245) 粉,14/15.4吋 磨砂PU手提筆電包 電腦包 附小包 (DH245) 粉,DICMI6,,,"{'M': 0, 'P': 664, 'Prime': ''}",0,{'B': '/items/DICMI6A9009XMU9/000001_155539716...,...,,,,筆電包^電腦包,內置超柔絨毛材質 保護筆電不刮傷\r\n柔軟細膩磨砂面料\r\n可隱藏式提把 可搭配行李提把,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,19463213,DCBB5U-A9008Q556,【NB】氣壓式站坐交替筆電架/FB17,【NB】氣壓式站坐交替筆電架/FB17,DCBB5U,,,"{'M': 5590, 'P': 3265, 'Prime': ''}",0,{'B': '/items/DCBB5UA9008Q556/000001_151635203...,...,,,非人為因素損壞提供一年保固,筆電架^NB,站坐交替 讓身體動起來\r\n適用於任何款式比電\r\n氣彈式自由懸停昇降系統\r\n距離桌...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
974,24360647,DHAFK9-1900AIYNY,【ASUS 直升12G超值組】X509JB 15.6吋窄邊雙碟筆電-星空灰 (i5-1035...,"<font color=""RED"">【ASUS好康送8G】</font><br><font ...",DHAFK9,,,"{'M': 0, 'P': 22900, 'Prime': ''}",0,{'B': '/items/DHAFK91900AIYNY/000001_158277524...,...,,,,,<a href=http://shopping.pchome.com.tw/DHAFIX-1...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
975,24644541,DHAFKI-A900AMCDV,ASUS VivoBook S15 S533FL-0088W10510U 幻彩白(i7-10...,<font color=#5c18d8>★最新10代新機上市</font><br>ASUS ...,DHAFKI,,,"{'M': 38900, 'P': 35900, 'Prime': ''}",0,{'B': '/items/DHAFKIA900AMCDV/000001_158763544...,...,包包 滑鼠,"<font color=""RED"">此機種無支援指紋辨識 沒有光碟機 購買前請留意</fon...",<B>華碩筆記型電腦兩年保固</B><BR>台灣區自2002年2月1日起凡購買華碩筆記型電腦...,S15^S532FL,<font color=#663399><b>《10代i7★限時狂降》\r\n網路價$389...,,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
976,19232724,DCBB0C-A9008MB3R,Polaris 旋轉式 鋁合金 筆電架 17吋(含)以下適用,Polaris 旋轉式 鋁合金 筆電架 17吋(含)以下適用 ( Alpha-06 ),DCBB7J,,,"{'M': 1690, 'P': 1390, 'Prime': ''}",0,{'B': '/items/DCBB0CA9008MB3R/000001_151321945...,...,,◆依照消費者保護法的規定，享有商品商品鑑賞期並非試用期，請留意，您所退回的商品必須回復原狀（...,保固一年,筆電架^筆電散熱架,"<font color=red><b>* 請搭配外接式鍵盤滑鼠一起使用, 以達到人體工學產品...",,,,,"{""AP1"":"""",""AP2"":"""",""AP3"":"""",""AP4"":"""",""AP5"":"""",..."
